### Get All Locations

In [ ]:
import pandas as pd
from flair.data import Sentence, Token
from flair.models import SequenceTagger

books = ['Outlander', 'Dragonfly in Amber', 'Voyager', 'Drums of Autumn', 'The Fiery Cross',
         'A Breath of Snow and Ashes',
         'An Echo in the Bone', 'Written in My Own Heart’s Blood']
extras = ['Other Books by this Author', 'About the Author']
bookstarts = [50, 17287, 37378, 61857, 89432, 119494, 152540, 177800, 202059]
f = open("data.txt", "r", encoding="utf8")
booknum = 3

book = ''
i = 0
for line in f:
    if i < bookstarts[booknum] and i > bookstarts[booknum - 1]:
        book = book + line
    i = i + 1
print(len(book))

# load tagger
tagger_ner = SequenceTagger.load("flair/ner-english-ontonotes-fast")
tagger_pos = SequenceTagger.load("flair/pos-english")


def ner_tag(text):
    sentence_ner = Sentence(text)

    # predict NER tags
    tagger_ner.predict(sentence_ner)
    return sentence_ner


def pos_tag(text):
    sentence_pos = Sentence(text.split(' '), use_tokenizer=True)

    # predict NER tags
    tagger_pos.predict(sentence_pos)
    return sentence_pos


def get_person_before(sentence_ner, token):
    if token.text == 'it':
        return token.text
    if token.text == 'I' or token.text == 'me':
        return 'Claire'
    prev_person = ''
    for entity in sentence_ner.get_spans('ner'):
        if entity.tag == 'PERSON' and entity.start_position < token.start_position:
            prev_person = entity.text
        elif entity.start_position > token.start_position:
            break
    return prev_person


import re


def split_into_chapters(booknum, text):
    if booknum < 7:
        return re.split(r'(\\n[0-9]+\\n\\n)', book)
    else:
        # needs refinement
        return re.split(r'[A-Z ]+\\n\\n', book)


def split_into_paragraphs(text):
    splitted = re.split('(\*\*\*)', text)
    return [x for x in splitted if x != '']


chapters = re.split(r'\n[0-9]+\n\n')

### From loc get lon and lat

In [47]:
f = open("locs_unique.txt", "r", encoding="utf8")

locs = []
for line in f:
    locs.append(line)

In [4]:
len(locs)

839

In [30]:
with open('locs_unique.txt', 'a') as f:
    for l in locs:
        f.write(l)

In [6]:
!pip install geopy

     -------------------------------------- 119.8/119.8 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00


In [48]:
from geopy.geocoders import Nominatim
import pandas as pd

geolocator = Nominatim(user_agent="geoapiExercises")
locs_dict = pd.DataFrame(columns = ['Name', 'Longitude', 'Latitude'])
i = 0
for l in locs:
    lat = 'None'
    lon = 'None'
    print(l)
    location = geolocator.geocode(l)
    if location is not None:
        lat = location.latitude
        lon = location.longitude
    locs_dict = locs_dict.append({'Name' : l, 'Longitude' : lon, 'Latitude' : lat}, ignore_index = True)


North Caicos

Netherlands

Philadelphia

Falkirk

Broch Tuarach

Whitehall

Cumberland

Oneida

Tsatsa'wi Tsatsa'wi

Redondo Beach

St. Denys

Florida

St. Ninian

Rue de Varennes

Royal Colony of South Carolina

Comte de Palles

Bois de Boulogne

City of Savannah


In [59]:
locs_dict.to_csv('locs_lonlat1.csv')

In [58]:
locs_dict

,Name,Longitude,Latitude
0,Richmond,-77.43428,37.5385087
1,Oak Grove,-90.5688924,41.4136501
2,Gwyllyn,None,None
3,Marblehead,-70.8578253,42.500096
4,Jocasta,7.614895275343744,44.37369555
...,...,...,...
833,Rue de Varennes,2.5048599,50.0351489
834,Royal Colony of South Carolina,None,None
835,Comte de Palles,15.0865574,62.4011503
836,Bois de Boulogne,2.248257867854371,48.86409285
